In [19]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import torch
from torch import Tensor
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
# from pytorch_loader import ImagesLoader
import matplotlib.pyplot as plt
import pandas as pd
import os
import torch
from torch.utils.data import Dataset
from skimage import io
from sklearn.metrics import accuracy_score, precision_score, recall_score
import torchvision.transforms as transforms


[nltk_data] Downloading package punkt to /Users/paddy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/paddy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:

products = '/Users/paddy/Desktop/facebook_products.csv'

df =  pd.read_csv(products)
df


,Unnamed: 0,id,product_name,category,product_description,price,location,url,page_id,create_time
0,0,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",£5.00,"Wokingham, Berkshire",https://www.gumtree.com/p/mirrors-clocks-ornam...,1.426705e+09,2022-02-26
1,1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,£20.00,"Inverness, Highland",https://www.gumtree.com/p/other-household-good...,1.426705e+09,2022-02-26
2,2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,£20.00,"Skegness, Lincolnshire",https://www.gumtree.com/p/outdoor-settings-fur...,1.426705e+09,2022-02-26
3,3,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,£115.00,"Radstock, Somerset",https://www.gumtree.com/p/other-dining-living-...,1.426705e+09,2022-02-26
4,4,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost | in ...,"Home & Garden / Dining, Living Room Furniture ...",New Design Shannon Corner sofa 5 Seater Avail...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
11313,7151,c4148656-78a9-4f3e-b393-134fdc5ef900,Sony PlayStation VR Move Bundle | in Acocks Gr...,Video Games & Consoles / Consoles / PS4 (Sony ...,Sony PlayStation VR Move Bundle353CASH ON COLL...,£260.00,"Acocks Green, West Midlands",https://www.gumtree.com/p/ps4/sony-playstation...,1.422159e+09,2022-02-28
11314,7152,564e3411-768d-4250-a624-b119d696f103,"Playstation VR V2 Bundle | in Acocks Green, We...",Video Games & Consoles / Consoles / PS4 (Sony ...,Playstation VR V2 Bundle355CASH ON COLLECTION ...,£235.00,"Acocks Green, West Midlands",https://www.gumtree.com/p/ps4/playstation-vr-v...,1.422159e+09,2022-02-28
11315,7153,2b0a652b-46a2-4297-b619-5efeeb222787,"Oculus quest 2 256gb | in Montrose, Angus | Gu...",Video Games & Consoles / Other Video Games & C...,Pick up only £250Comes with two pistols stocks...,£250.00,"Montrose, Angus",https://www.gumtree.com/p/other-video-games/oc...,1.426669e+09,2022-02-28
11316,7154,719fd40a-870e-4144-b324-55dff2e66fb4,Logitech driving force shifter | in Carrickfer...,Video Games & Consoles / Video Game Accessorie...,Bought at christmas from currys retailing at £...,£30.00,"Carrickfergus, County Antrim",https://www.gumtree.com/p/other-video-game-acc...,1.426700e+09,2022-02-28


In [21]:
df = df.drop(df.columns[0], axis=1)
df['product_name'] = df['product_name'].str.encode('ascii', 'ignore').str.decode('ascii')
df['product_description'] = df['product_description'].str.encode('ascii', 'ignore').str.decode('ascii')
df.head()

,id,product_name,category,product_description,price,location,url,page_id,create_time
0,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",£5.00,"Wokingham, Berkshire",https://www.gumtree.com/p/mirrors-clocks-ornam...,1.426705e+09,2022-02-26
1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richards (model no 48755)Stainless stee...,£20.00,"Inverness, Highland",https://www.gumtree.com/p/other-household-good...,1.426705e+09,2022-02-26
2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I dont ...,£20.00,"Skegness, Lincolnshire",https://www.gumtree.com/p/outdoor-settings-fur...,1.426705e+09,2022-02-26
3,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,£115.00,"Radstock, Somerset",https://www.gumtree.com/p/other-dining-living-...,1.426705e+09,2022-02-26
4,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost | in ...,"Home & Garden / Dining, Living Room Furniture ...",New Design Shannon Corner sofa 5 Seater Avail...,NaN,NaN,NaN,NaN,NaN


In [22]:
stop = stopwords.words('english')
df_nlp = df[['id', 'product_description']]
df_nlp['product_description'] = df_nlp['product_description'].astype(str)
df_nlp['product_description'] = df_nlp['product_description'].str.lower()
df_nlp['product_description'] = df_nlp['product_description'].str.replace('[^a-zA-Z ]', '')
df_nlp['product_description'] = df_nlp['product_description'].str.replace('[0-9]', '')
df_nlp['product_description'] = df_nlp['product_description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_nlp.head()

/var/folders/1j/z792ghj15kz_y3xd3w6q88d00000gn/T/ipykernel_43699/236711266.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['product_description'] = df_nlp['product_description'].astype(str)
/var/folders/1j/z792ghj15kz_y3xd3w6q88d00000gn/T/ipykernel_43699/236711266.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['product_description'] = df_nlp['product_description'].str.lower()
/var/folders/1j/z792ghj15kz_y3xd3w6q88d00000gn/T/ipykernel_43699/236711266.py:5: FutureWarning: The default va

,id,product_description
0,243809c0-9cfc-4486-ad12-3b7a16605ba9,mirror wall art posted nisha dining living roo...
1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,morphy richards model stainless steel tier sta...
2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,collection dont drive ono
3,59948726-29be-4b35-ade5-bb2fd7331856,great reclaimed army ammunition box used coffe...
4,16dbc860-696e-4cda-93f6-4dd4926573fb,new design shannon corner sofa seater available


In [23]:
full_text_list = []


desc_list = [[x] for x in df_nlp['product_description']]
# print(desc_list)

# corpus_text = 'n'.join(desc_list)


data = []
temp = []
for desc in desc_list:
    for index, item in enumerate((desc)):
        # print(item)
        for i in sent_tokenize(item):
            temp = []
            for j in word_tokenize(i):
                temp.append(j)
            data.append(temp)

print(data[0])



'''
two dfs,


word embedding df:
takes wach word & will be constructed into 128 row vectorisation of that word via the word2vec model


conext df: takes each word in column a
in column b takes worse that surround that workd from (word-2 --> word --> word+2)
so for every word we have four rows, however we also need to add negative samples to each


as we train the model the model predicts the context words that would go either sides of the chosen word

each word embedding should be unique but the context words should stay same size even if duplicates as were training multiple possible words that match the current word


'''

['mirror', 'wall', 'art', 'posted', 'nisha', 'dining', 'living', 'room', 'furniture', 'mirrors', 'clocks', 'ornaments', 'wokingham', 'february', 'size', 'xcm']


'\ntwo dfs,\n\n\nword embedding df:\ntakes wach word & will be constructed into 128 row vectorisation of that word via the word2vec model\n\n\nconext df: takes each word in column a\nin column b takes worse that surround that workd from (word-2 --> word --> word+2)\nso for every word we have four rows, however we also need to add negative samples to each\n\n\nas we train the model the model predicts the context words that would go either sides of the chosen word\n\neach word embedding should be unique but the context words should stay same size even if duplicates as were training multiple possible words that match the current word\n\n\n'

In [24]:
final_dataset = []
window_size = 3
negative_samples_size = 2
full_data = data[0:10]
# print(data)

def embedding_prep(description):
    for index, centre_word in enumerate(description[window_size -3 : -window_size]):
        print(centre_word)
        # if index == 0 or index == 1:
        #     context_words = [context_word for context_word in description[index : index + 2] if context_word != centre_word]
        context_words = [context_word for context_word in description[index : index + 2 * window_size - 1] if context_word != centre_word]

        for context_word in context_words:

            final_dataset.append([centre_word, context_word, 1])
            if context_word in context_word:
                # print(context_word)
                try:

                    negative_samples = np.random.choice([w for w in description[window_size-3:-window_size] if w != centre_word and w not in context_words], negative_samples_size)
                except:
                    continue
                
                    

        
            for negative_samp in negative_samples:
            
            #add a training row
                final_dataset.append([centre_word, negative_samp, 0])
    print(final_dataset)





for index, data in enumerate(full_data):
    embedding_prep(data)





mirror
wall
art
posted
nisha
dining
living
room
furniture
mirrors
clocks
ornaments
wokingham
[['mirror', 'wall', 1], ['mirror', 'room', 0], ['mirror', 'living', 0], ['mirror', 'art', 1], ['mirror', 'ornaments', 0], ['mirror', 'furniture', 0], ['mirror', 'posted', 1], ['mirror', 'dining', 0], ['mirror', 'room', 0], ['mirror', 'nisha', 1], ['mirror', 'ornaments', 0], ['mirror', 'ornaments', 0], ['wall', 'art', 1], ['wall', 'room', 0], ['wall', 'ornaments', 0], ['wall', 'posted', 1], ['wall', 'furniture', 0], ['wall', 'room', 0], ['wall', 'nisha', 1], ['wall', 'ornaments', 0], ['wall', 'living', 0], ['wall', 'dining', 1], ['wall', 'wokingham', 0], ['wall', 'ornaments', 0], ['art', 'posted', 1], ['art', 'furniture', 0], ['art', 'mirrors', 0], ['art', 'nisha', 1], ['art', 'room', 0], ['art', 'wall', 0], ['art', 'dining', 1], ['art', 'mirror', 0], ['art', 'mirror', 0], ['art', 'living', 1], ['art', 'mirrors', 0], ['art', 'room', 0], ['posted', 'nisha', 1], ['posted', 'art', 0], ['posted', 'o

In [25]:
df = pd.DataFrame(columns=['center_word', 'context_word', 'label'], data=final_dataset)
words = np.intersect1d(df.context_word, df.center_word)
df = df[(df.center_word.isin(words)) & (df.context_word.isin(words))].reset_index(drop=True)
df

,center_word,context_word,label
0,mirror,wall,1
1,mirror,room,0
2,mirror,living,0
3,mirror,art,1
4,mirror,ornaments,0
...,...,...,...
1031,corner,shannon,0
1032,corner,new,0
1033,corner,shannon,0
1034,corner,shannon,0


In [26]:
model = Word2Vec(sentences=final_dataset, vector_size=100, window=5, min_count=1, workers=4)

class word2vec(nn.module):


SyntaxError: unexpected EOF while parsing (4232219403.py, line 3)